In [2]:
"""
This document seeks to implement polynomial multiplcation using the NTT (number theoretic transform) as 

NTT^{-1}(NTT(f(x)) \circ NTT(g(x)))

so we are just doing the forward NTT on each polynomial, performing the convolution, then taking the inverse.
""";

In [1]:
# Finite field setup
p = 17
F = GF(p)         # Finite field modulo p
n = 8             # Size of the transform
g = F(3)          # Primitive root of unity

# Compute n-th root of unity
omega = g**((p-1)//n)

# Polynomial coefficients as elements of the field
A = [F(1), F(2), F(3), F(4), F(0), F(0), F(0), F(0)]

# Forward transform using DFT
def ntt(a, omega, n):
    return [sum(a[j] * omega**(j*k) for j in range(n)) for k in range(n)]

# Inverse transform
def intt(a, omega, n):
    omega_inv = omega**-1
    n_inv = F(n)**-1
    return [sum(a[j] * omega_inv**(j*k) for j in range(n)) * n_inv for k in range(n)]

# Perform forward and inverse transforms
A_ntt = ntt(A, omega, n)
A_recovered = intt(A_ntt, omega, n)

print("Original:", A)
print("NTT:", A_ntt)
print("Recovered:", A_recovered)


Original: [1, 2, 3, 4, 0, 0, 0, 0]
NTT: [10, 16, 6, 11, 15, 13, 7, 15]
Recovered: [1, 2, 3, 4, 0, 0, 0, 0]


In [5]:
from sympy.discrete.transforms import ntt
from sympy import primerange

# Example parameters
seq = [1, 2, 3, 4, 0, 0, 0, 0]  # Input sequence (polynomial coefficients)
prime = 17  # A prime number

# Perform the Number Theoretic Transform
result = ntt(seq, prime)

print("Original sequence:", seq)
print("NTT result:", result)

Original sequence: [1, 2, 3, 4, 0, 0, 0, 0]
NTT result: [10, 16, 6, 11, 15, 13, 7, 15]
